In [ ]:
''' 
0) Fix at_risk_value...right now CAD and USD positions are all being summed up. they need to be converted to the base currency first.
1) Calculate current amount_invested, portfolio value, current_leverage_multiplier, current liquidation value, current_return_pct, current_profit, at_risk_Value, at_risk_as_a_pct_of_current_liquidation_value
2) Run a strategy and figure out ideal portfolio
3) calculate the difference between the current portfolio and the ideal portfolio
4) calculate the delta for the portfolio and a list of orders to place.
5) Place the orders (with approvals of the user) (if the user says no, then the orders are not placed)
'''

In [1]:
import nest_asyncio
import json
from pprint import pprint
from ib_insync import IB, util, Stock, Order
nest_asyncio.apply()

In [7]:
trader.disconnect()

Now Disconnecting from IBKR Trader Workstation.....


In [8]:
class IBKRTrader:
    def __init__(self, update_stoplosses=False):
        # Connect to IBKR Trader Workstation
        self.ib = IB()
        self.connect()
        self.account_info = self.update_account_info()
        self.exchange_rate = self.get_exchange_rate()
        self.positions = self.current_positions()
        self.portfolio = self.current_portfolio()
        self.open_orders = self.get_open_orders()
        self.at_risk_portfolio = self.calculate_current_risk_portfolio()
        self.value_at_risk = sum(self.at_risk_portfolio.values())
        if update_stoplosses==True:
            self.update_stop_losses(risk_pct=0.10, max_acceptable_risk_deviation=0.2, force_update_stoplosses=False)
        pprint({'account_info': self.account_info,
            'value_at_risk': round(self.value_at_risk, 1),
            # 'Worst_case_scenario_NetLiquidation_value': int(self.account_info['NetLiquidation']) - self.value_at_risk
            })
        
    def connect(self):
        # Load the configuration file
        config = {'port':7777, 'host':'localhost'}
        # Connect to TWS
        self.ib.connect(config['host'], config['port'], clientId=1)
        print("Now Connecting to IBKR Trader Workstation.....")
    def disconnect(self):
        # Disconnect from TWS
        print("Now Disconnecting from IBKR Trader Workstation.....")
        self.ib.disconnect()
    def current_positions(self):
        # Get current positions
        self.positions = self.ib.positions()
        return self.positions
    def current_portfolio(self):
        # Get current portfolio value
        self.portfolio = self.ib.portfolio()
        return self.portfolio
    def get_open_orders(self):
        self.open_orders = self.ib.reqAllOpenOrders()
        return self.open_orders
    def calculate_current_risk_portfolio(self):
        self.portfolio = self.current_portfolio()
        self.open_orders = self.get_open_orders()
        at_risk_portfolio = {}
        for myposition in self.portfolio:
            symbol = myposition.contract.symbol
            primaryExchange = myposition.contract.primaryExchange
            currency = myposition.contract.currency
            stoploss_order = None
            for open_order in self.open_orders:
                if open_order.contract.symbol == symbol and open_order.order.orderType == 'STP':
                    stoploss_order = open_order
                    break
            
            if stoploss_order is None:
                market_value = myposition.position * myposition.marketPrice
                at_risk_portfolio[symbol] = market_value
            else:
                market_value = myposition.position * myposition.marketPrice
                stoploss_value = open_order.order.totalQuantity * open_order.order.auxPrice
                at_risk_value = market_value - stoploss_value
                at_risk_portfolio[symbol] = at_risk_value
                # print(f"Symbol: {symbol}, Market Value: {market_value}, Stop Loss Value: {stoploss_value}, At Risk Value: {at_risk_value}")
        
        self.at_risk_portfolio = at_risk_portfolio
        self.at_risk_value = sum(self.at_risk_portfolio.values())
        return self.at_risk_portfolio
    def place_stop_loss_order(self, direction, symbol, quantity, stop_price, primaryExchange, currency):
        # Define the contract for the symbol
        contract = Stock(symbol=symbol, exchange='SMART', currency=currency, primaryExchange=primaryExchange)

        # Create a stop-loss order
        stop_loss_order = Order(
            action=direction,           # 'SELL' to close or reduce a long position
            totalQuantity=quantity,     # Number of shares
            orderType='STP',            # Stop order
            auxPrice=stop_price,        # Stop price
            tif='GTC'                   # Good-till-canceled
        )

        # Place the order and return the order ID
        trade = self.ib.placeOrder(contract, stop_loss_order)
        self.ib.sleep(2)  # Wait for the order to be processed
        return trade.order.orderId
    def update_stop_losses(self, risk_pct=0.10, max_acceptable_risk_deviation=0.2, force_update_stoplosses=False):
        for myposition in self.portfolio:
            symbol = myposition.contract.symbol
            primaryExchange = myposition.contract.primaryExchange
            currency = myposition.contract.currency
            stoploss_order = None
            for open_order in self.open_orders:
                if open_order.contract.symbol == symbol and open_order.order.orderType == 'STP':
                    stoploss_order = open_order
                    break
            if stoploss_order is None:
                quantity = int(myposition.position)
                current_price = myposition.marketPrice
                stop_price = round(current_price * (1-risk_pct), 1)
                direction = 'SELL' if quantity > 0 else 'BUY'
                print(f"Placing a stop loss order with the following parameters: symbol:{symbol}, qty: {quantity}, current_price: {current_price}, stop_price: {stop_price}, direction: {direction}")
                input("Press Enter to continue...")
                order_id = self.place_stop_loss_order(direction, symbol, quantity, stop_price, primaryExchange, currency)
                print(f"Stop loss order placed with order ID: {order_id}")
            else:
                # check if stoploss is as per the risk profile.
                current_stoploss_price = stoploss_order.order.auxPrice
                current_stoploss_qty = int(stoploss_order.order.totalQuantity)
                current_position_qty = int(myposition.position)
                current_price = myposition.marketPrice
                direction = 'SELL' if current_position_qty > 0 else 'BUY'
                stoploss_pct_theshold = 1 - ((risk_pct * max_acceptable_risk_deviation) + risk_pct) if direction == 'SELL' else 1 + ((risk_pct * max_acceptable_risk_deviation) + risk_pct)
                stoploss_price_threshold = current_price * stoploss_pct_theshold
                
                if current_stoploss_qty != current_position_qty or force_update_stoplosses==True:
                    quantity = int(myposition.position)
                    current_price = myposition.marketPrice
                    stop_price = round(current_price * (1-risk_pct), 1)
                    print(f"Updating a stop loss order with the following parameters: symbol:{symbol}, qty: {quantity}, current_price: {current_price}, stop_price: {stop_price}, direction: {direction}")
                    input("Press Enter to continue...")
                    order_id = self.place_stop_loss_order(direction, symbol, quantity, stop_price, primaryExchange, currency)
                    print(f"Stop loss order updated with order ID: {order_id}")
                    self.ib.cancelOrder(stoploss_order.order)
                    print(f"Previous Stop loss order with order ID: {stoploss_order.order.orderId} is cancelled.")

                elif (direction == 'SELL' and current_stoploss_price < stoploss_price_threshold) or (direction == 'BUY' and current_stoploss_price > stoploss_price_threshold):
                    quantity = int(myposition.position)
                    current_price = myposition.marketPrice
                    stop_price = round(current_price * (1-risk_pct), 1)
                    print(f"Updating a stop loss order with the following parameters: symbol:{symbol}, qty: {quantity}, current_price: {current_price}, old_stop_price: {current_stoploss_price}, new_stop_price: {stop_price}, direction: {direction}")
                    input("Press Enter to continue...")
                    order_id = self.place_stop_loss_order(direction, symbol, quantity, stop_price, primaryExchange, currency)
                    print(f"Stop loss order updated with order ID: {order_id}")
                    self.ib.cancelOrder(stoploss_order.order)
                    print(f"Previous Stop loss order with order ID: {stoploss_order.order.orderId} is cancelled.")
                else:
                    # if force_update_stoplosses==True:
                    print(f"Stop loss order for symbol: {symbol} is already at an acceptable level.")
                    
        # update open orders
        self.open_orders = self.get_open_orders()
        # update at_risk_portfolio
        self.at_risk_portfolio = self.calculate_current_risk_portfolio()
        self.at_risk_value = sum(self.at_risk_portfolio.values())
    def update_account_info(self):
        account_summary = self.ib.accountSummary()
        account_info = {}
        for item in account_summary:
            if item.tag == 'NetLiquidation':
                account_info['NetLiquidation'] = str(item.currency) + ' ' + str(item.value)
            if item.tag == 'GrossPositionValue':
                account_info['GrossPositionValue'] = str(item.currency) + ' ' + str(item.value)
        account_info['current_leverage_multiplier'] = round(float(account_info['GrossPositionValue'].split(' ')[1]) / float(account_info['NetLiquidation'].split(' ')[1]), 2)
            
        self.account_info = account_info
        return self.account_info
    def get_exchange_rate(self):
        account_summary = self.ib.accountSummary()
        for item in account_summary:
            if item.tag == 'ExchangeRate':
                self.exchange_rate = float(item.value)
                break

In [9]:
# Create an instance of the IBKRTrader class
trader = IBKRTrader(update_stoplosses=False)

Now Connecting to IBKR Trader Workstation.....
{'account_info': {'GrossPositionValue': 'CAD 715540.54',
                  'NetLiquidation': 'CAD 397761.15',
                  'current_leverage_multiplier': 1.8},
 'value_at_risk': 30660.1}


Peer closed connection.


In [10]:
trader.update_stop_losses(risk_pct=0.075, max_acceptable_risk_deviation=0.1, force_update_stoplosses=False)

Stop loss order for symbol: AAAU is already at an acceptable level.
Stop loss order for symbol: DELL is already at an acceptable level.
Stop loss order for symbol: META is already at an acceptable level.
Stop loss order for symbol: NVDA is already at an acceptable level.
Stop loss order for symbol: PANW is already at an acceptable level.
Stop loss order for symbol: QCOM is already at an acceptable level.
Stop loss order for symbol: SPOT is already at an acceptable level.


In [23]:
rate = trader.get_exchange_rate()
rate

In [9]:
positions = trader.current_positions()
portfolio = trader.current_portfolio()
open_orders = trader.get_open_orders()

In [10]:
portfolio

[PortfolioItem(contract=Stock(conId=458886642, symbol='AAAU', right='0', primaryExchange='BATS', currency='USD', localSymbol='AAAU', tradingClass='AAAU'), position=2000.0, marketPrice=23.17480085, marketValue=46349.6, averageCost=23.655, unrealizedPNL=-960.4, realizedPNL=0.0, account='U13152795'),
 PortfolioItem(contract=Stock(conId=4205, symbol='AEM', right='0', primaryExchange='NYSE', currency='USD', localSymbol='AEM', tradingClass='AEM'), position=500.0, marketPrice=68.19, marketValue=34095.0, averageCost=61.985, unrealizedPNL=3102.5, realizedPNL=0.0, account='U13152795'),
 PortfolioItem(contract=Stock(conId=5027019, symbol='CNQ', right='0', primaryExchange='TSE', currency='CAD', localSymbol='CNQ', tradingClass='CNQ'), position=300.0, marketPrice=103.68499755, marketValue=31105.5, averageCost=112.46, unrealizedPNL=-2632.5, realizedPNL=0.0, account='U13152795'),
 PortfolioItem(contract=Stock(conId=346218218, symbol='DELL', right='0', primaryExchange='NYSE', currency='USD', localSymbo

In [11]:
open_orders

[Trade(contract=Stock(conId=4815747, symbol='NVDA', right='?', exchange='SMART', currency='USD', localSymbol='NVDA', tradingClass='NMS'), order=Order(permId=225182841, action='SELL', totalQuantity=50.0, orderType='STP', lmtPrice=0.0, auxPrice=917.24, tif='GTC', ocaType=3, displaySize=2147483647, rule80A='0', trailStopPrice=917.24, openClose='', volatilityType=0, deltaNeutralOrderType='None', referencePriceType=0, account='U13152795', clearingIntent='IB', adjustedOrderType='None', cashQty=0.0, dontUseAutoPriceForHedge=True), orderStatus=OrderStatus(orderId=0, status='PreSubmitted', filled=0.0, remaining=50.0, avgFillPrice=0.0, permId=225182841, parentId=0, lastFillPrice=0.0, clientId=0, whyHeld='trigger', mktCapPrice=0.0), fills=[], log=[TradeLogEntry(time=datetime.datetime(2024, 5, 31, 3, 50, 51, 451397, tzinfo=datetime.timezone.utc), status='PreSubmitted', message='', errorCode=0), TradeLogEntry(time=datetime.datetime(2024, 5, 31, 3, 51, 53, 483870, tzinfo=datetime.timezone.utc), stat

Updating a stop loss order with the following parameters: symbol:AAAU, qty: 2000, current_price: 23.17480085, old_stop_price: 21.6, new_stop_price: 22.0, direction: SELL
Stop loss order updated with order ID: 35
Previous Stop loss order with order ID: 19 is cancelled.
Updating a stop loss order with the following parameters: symbol:AEM, qty: 500, current_price: 68.19, old_stop_price: 63.7, new_stop_price: 64.8, direction: SELL
Stop loss order updated with order ID: 36
Previous Stop loss order with order ID: 20 is cancelled.
Updating a stop loss order with the following parameters: symbol:CNQ, qty: 300, current_price: 103.68499755, old_stop_price: 94.5, new_stop_price: 98.5, direction: SELL
Stop loss order updated with order ID: 37
Previous Stop loss order with order ID: 21 is cancelled.
Updating a stop loss order with the following parameters: symbol:DELL, qty: 300, current_price: 139.6499939, old_stop_price: 131.4, new_stop_price: 132.7, direction: SELL
Stop loss order updated with or

Error 321, reqId 43: Error validating request.-'bG' : cause - The size value cannot be zero:
Canceled order: Trade(contract=Stock(symbol='SCCO', exchange='SMART', primaryExchange='NYSE', currency='USD'), order=Order(orderId=43, clientId=1, action='BUY', orderType='STP', auxPrice=112.6, tif='GTC'), orderStatus=OrderStatus(orderId=43, status='Cancelled', filled=0.0, remaining=0.0, avgFillPrice=0.0, permId=0, parentId=0, lastFillPrice=0.0, clientId=0, whyHeld='', mktCapPrice=0.0), fills=[], log=[TradeLogEntry(time=datetime.datetime(2024, 5, 31, 3, 54, 53, 384303, tzinfo=datetime.timezone.utc), status='PendingSubmit', message='', errorCode=0), TradeLogEntry(time=datetime.datetime(2024, 5, 31, 3, 54, 53, 391123, tzinfo=datetime.timezone.utc), status='Cancelled', message="Error 321, reqId 43: Error validating request.-'bG' : cause - The size value cannot be zero:", errorCode=321)], advancedError='')
